<a href="https://colab.research.google.com/github/chi-hun/chi-hun/blob/main/oof.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Out Of Fold**

In [ ]:
import pandas as pd
import numpy as np
import gc
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error, roc_auc_score
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow.keras import layers

# XGB

In [ ]:
train = pd.read_csv('/content/sample_data/california_housing_train.csv',)
test = pd.read_csv('/content/sample_data/california_housing_test.csv')
x_train = train.copy()
y_train = x_train.pop('median_house_value')
x_test = test.copy()
y_test = x_test.pop('median_house_value')

In [ ]:
folds = 5 # 나눌 폴드수
kfold = KFold(n_splits=folds, shuffle=True) # kfold 생성
xgb = XGBRegressor()
pre_all = 0 # 폴드별 예측값 합 저장 
for fold ,(train_idx, val_idx) in enumerate(kfold.split(x_train, y_train)):
    model = xgb.fit(x_train.iloc[train_idx], y_train.iloc[train_idx], eval_set=[(x_train.iloc[val_idx], y_train.iloc[val_idx])], eval_metric='rmse',verbose=100)# 폴드 훈련
    pre_v = model.predict(x_train.iloc[val_idx]) # 폴드 검증 예측
    rmse = mean_squared_error(y_train.iloc[val_idx], pre_v, squared=True) # 검증 평가
    sc = model.score(x_train.iloc[val_idx], y_train.iloc[val_idx]) # 모델 평가
    pre = model.predict(x_test) / folds # 테스트 자료의 예측값을 폴드수로 나누어 저장
    pre_all += pre
    del model # 모델 삭제
    gc.collect()
    print(f'======{fold+1} / {folds}======\n sc : {sc}, rmse : {rmse}')


[03:36:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:220405
[99]	validation_0-rmse:54227.8
======1 / 5======
 sc : 0.7865070620565007, rmse : 2940651836.1883554
[03:36:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:215534
[99]	validation_0-rmse:54306.2
======2 / 5======
 sc : 0.7771336575191564, rmse : 2949163875.0810614
[03:36:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:216117
[99]	validation_0-rmse:54197.1
======3 / 5======
 sc : 0.7809879240209472, rmse : 2937322138.489846
[03:36:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:213872
[99]	validation_0-rmse:55060.7
======4 / 5======
 sc : 0.7707934073512362,

In [ ]:
rmse_oof = mean_squared_error(y_test, pre_all, squared=True)

In [ ]:
model = xgb.fit(x_train, y_train)
pre = model.predict(x_test)
rmse_one = mean_squared_error(y_test, pre, squared=True)

[03:40:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
print(f'one : {rmse_one}, oof : {rmse_oof}, 차이 = {rmse_one - rmse_oof}')

one : 3095456066.206096, oof : 3088113156.788528, 차이 = 7342909.417568207


# deep learnig

In [ ]:
train = pd.read_csv('/content/sample_data/mnist_train_small.csv')
test = pd.read_csv('/content/sample_data/mnist_test.csv')

x_train = train.copy()
y_train = x_train.pop('6').values
x_train = x_train.values

x_test = test.copy()
y_test = x_test.pop('7').values
x_test = x_test.values

In [ ]:
def model_make():
    inputs = layers.Input(shape=(28*28))
    x = layers.Dense(1024, activation='relu')(inputs)
    x = layers.Dense(512, activation='relu')(x)
    outputs = layers.Dense(10, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    return model

In [ ]:
folds = 5
skf = StratifiedKFold(n_splits=folds, shuffle=True) # 폴드 생성
pre_all = np.zeros((x_test.shape[0], 10)) # 예측 결과 저장
for fold, (train_idx, val_idx) in enumerate(skf.split(x_train, y_train)):
    tf.keras.backend.clear_session() # 모델 삭제
    gc.collect()

    print(f'fold : {fold+1} / {folds}')
    model = model_make() # 모델 생성
    model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
    his = model.fit(x_train[train_idx], y_train[train_idx], validation_data=(x_train[val_idx], y_train[val_idx]), epochs=5)
    best_accuracy = np.array(his.history['val_accuracy']).max() # 최고정확도
    pre = model.predict(x_test) # 테스트 데이터 예측
    pre_all += pre / folds # 폴드수로 나눈 예측값 저장
    print(f'best_accuracy : {best_accuracy}')

fold : 1 / 5
Epoch 1/5
500/500 [==============================] - 9s 17ms/step - loss: 4.3608 - accuracy: 0.8677 - val_loss: 0.7069 - val_accuracy: 0.9003
Epoch 2/5
500/500 [==============================] - 9s 18ms/step - loss: 0.4113 - accuracy: 0.9346 - val_loss: 0.5021 - val_accuracy: 0.9280
Epoch 3/5
500/500 [==============================] - 9s 17ms/step - loss: 0.2723 - accuracy: 0.9499 - val_loss: 0.5050 - val_accuracy: 0.9300
Epoch 4/5
500/500 [==============================] - 9s 17ms/step - loss: 0.2486 - accuracy: 0.9507 - val_loss: 0.4478 - val_accuracy: 0.9340
Epoch 5/5
313/313 [==============================] - 2s 5ms/step
best_accuracy : 0.9417499899864197
fold : 2 / 5
Epoch 1/5
500/500 [==============================] - 10s 19ms/step - loss: 3.5825 - accuracy: 0.8612 - val_loss: 0.4969 - val_accuracy: 0.9018
Epoch 2/5
500/500 [==============================] - 9s 17ms/step - loss: 0.3354 - accuracy: 0.9324 - val_loss: 0.4790 - val_accuracy: 0.8978
Epoch 3/5
500/500 [==

In [ ]:
pre_oof = np.argmax(pre_all, axis=1)
tr = (y_test == pre_oof).sum()
print(f'정확도 : {round(tr / len(pre_oof), 2)*100}%')

정확도 : 97.0%


In [ ]:
model = model_make()
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
his = model.fit(x_train, y_train, validation_split=0.1, epochs=5)
pre = model.predict(x_test)
pre_one = np.argmax(pre, axis=1)
tr = (y_test == pre_one).sum()
print(f'정확도 : {round(tr / len(pre_oof), 2)*100}%')

Epoch 1/5
563/563 [==============================] - 11s 19ms/step - loss: 4.1753 - accuracy: 0.8697 - val_loss: 0.5973 - val_accuracy: 0.9080
Epoch 2/5
563/563 [==============================] - 11s 19ms/step - loss: 0.3615 - accuracy: 0.9348 - val_loss: 0.4644 - val_accuracy: 0.9235
Epoch 3/5
563/563 [==============================] - 10s 18ms/step - loss: 0.2428 - accuracy: 0.9502 - val_loss: 0.4203 - val_accuracy: 0.9300
Epoch 4/5
563/563 [==============================] - 9s 16ms/step - loss: 0.1689 - accuracy: 0.9614 - val_loss: 0.3635 - val_accuracy: 0.9310
Epoch 5/5
313/313 [==============================] - 2s 5ms/step
정확도 : 94.0%
